# --- Day 7: Camel Cards ---
Your all-expenses-paid trip turns out to be a one-way, five-minute ride in an airship. (At least it's a cool airship!) It drops you off at the edge of a vast desert and descends back to Island Island.

"Did you bring the parts?"

You turn around to see an Elf completely covered in white clothing, wearing goggles, and riding a large camel.

"Did you bring the parts?" she asks again, louder this time. You aren't sure what parts she's looking for; you're here to figure out why the sand stopped.

"The parts! For the sand, yes! Come with me; I will show you." She beckons you onto the camel.

After riding a bit across the sands of Desert Island, you can see what look like very large rocks covering half of the horizon. The Elf explains that the rocks are all along the part of Desert Island that is directly above Island Island, making it hard to even get there. Normally, they use big machines to move the rocks and filter the sand, but the machines have broken down because Desert Island recently stopped receiving the parts they need to fix the machines.

You've already assumed it'll be your job to figure out why the parts stopped when she asks if you can help. You agree automatically.

Because the journey will take a few days, she offers to teach you the game of Camel Cards. Camel Cards is sort of similar to poker except it's designed to be easier to play while riding a camel.

In Camel Cards, you get a list of hands, and your goal is to order them based on the strength of each hand. A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2. The relative strength of each card follows this order, where A is the highest and 2 is the lowest.

Every hand is exactly one type. From strongest to weakest, they are:

- **Five of a kind**, where all five cards have the same label: AAAAA
- **Four of a kind**, where four cards have the same label and one card has a different label: AA8AA
- **Full house**, where three cards have the same label, and the remaining two cards share a different label: 23332
- **Three of a kind**, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
- **Two pair**, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
- **One pair**, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
- **High card**, where all cards' labels are distinct: 23456

Hands are primarily ordered based on type; for example, every full house is stronger than any three of a kind.

If two hands have the same type, a second ordering rule takes effect. Start by comparing the first card in each hand. If these cards are different, the hand with the stronger first card is considered stronger. If the first card in each hand have the same label, however, then move on to considering the second card in each hand. If they differ, the hand with the higher second card wins; otherwise, continue with the third card in each hand, then the fourth, then the fifth.

So, 33332 and 2AAAA are both four of a kind hands, but 33332 is stronger because its first card is stronger. Similarly, 77888 and 77788 are both a full house, but 77888 is stronger because its third card is stronger (and both hands have the same first and second card).

To play Camel Cards, you are given a list of hands and their corresponding bid (your puzzle input). For example:

`32T3K 765`

`T55J5 684`

`KK677 28`

`KTJJT 220`

`QQQJA 483`

This example shows five hands; each hand is followed by its bid amount. Each hand wins an amount equal to its bid multiplied by its rank, where the weakest hand gets rank 1, the second-weakest hand gets rank 2, and so on up to the strongest hand. Because there are five hands in this example, the strongest hand will have rank 5 and its bid will be multiplied by 5.

So, the first step is to put the hands in order of strength:

- 32T3K is the only one pair and the other hands are all a stronger type, so it gets rank 1.
- KK677 and KTJJT are both two pair. Their first cards both have the same label, but the second card of KK677 is stronger (K vs T), so KTJJT gets rank 2 and KK677 gets rank 3.
- T55J5 and QQQJA are both three of a kind. QQQJA has a stronger first card, so it gets rank 5 and T55J5 gets rank 4.

Now, you can determine the total winnings of this set of hands by adding up the result of multiplying each hand's bid with its rank (765 * 1 + 220 * 2 + 28 * 3 + 684 * 4 + 483 * 5). So the total winnings in this example are 6440.

Find the rank of every hand in your set. What are the total winnings?

In [2]:
def getHands():
    with open("hands.txt") as file:
        return file.read()

In [3]:
#formatting
hands = getHands().split("\n")
hands = [x.split() for x in hands]
cards = [x[0] for x in hands] #Just keeps track of the cards (letters)
bids = [int(x[1]) for x in hands] #Just keeps track of the bids
handDict = dict(zip(cards, bids))

#Keeps track of card values
cardValues = dict({
    "A":14,
    "K":13,
    "Q":12,
    "J":11,
    "T":10,
    "9":9,
    "8":8,
    "7":7,
    "6":6,
    "5":5,
    "4":4,
    "3":3,
    "2":2
})

#Hand rankings (0:high card, 1:one pair, 2:two pair, 3:three of a kind, 4:full house, 5:four of a kind, 6:five of a kind)
handTypes = [[], [], [], [], [], [], []]

#This sorts each 
for hand in cards:
    #Check for high card
    if len(set(hand)) == 5:
        handTypes[0].append(hand)
        
    #Checks for one pair
    elif len(set(hand)) == 4:
        handTypes[1].append(hand)
        
    #Checks for two pair and three of a kind
    elif len(set(hand)) == 3:
        
        #Counts the occurences of each card in hand
        cardCount = dict()
        for card in hand:
            if card not in cardCount.keys():
                cardCount[card] = 1
            else:
                cardCount[card] += 1
                
        #If a card does not occur 3 times then it is a two pair
        if 3 not in cardCount.values():
            handTypes[2].append(hand)
            continue
        #Otherwise it is a three of a kind
        handTypes[3].append(hand)
        
    #Checks for full house and four of a kind
    elif len(set(hand)) == 2:
        
        #Counts the occurences of each card in hand
        cardCount = dict()
        for card in hand:
            if card not in cardCount.keys():
                cardCount[card] = 1
            else:
                cardCount[card] += 1
                
        #If there are not 4 of one card type in the hand, then it's a full house
        if 4 not in cardCount.values():
            handTypes[4].append(hand)
            continue
        #Otherwise it's a four of a kind
        handTypes[5].append(hand)
        
    #Checks for five of a kind
    elif len(set(hand)) == 1:
        handTypes[6].append(hand)
        
#Gets a list of all hands in order
rankedHands = []
for i in handTypes:
    #For every hand type, add all hands to ranked hands after sorting them by their card value
    rankedHands.extend(sorted(i, key=lambda cards: [cardValues[x] for x in cards]))
    
#Get the total winnings by multiplying the bet and rank for each hand in rankedHands and getting the sum
totalWinnings = sum([handDict[hand] * rank for rank, hand in enumerate(rankedHands, 1)])
print(f"Total winnings: {totalWinnings}")

Total winnings: 250474325


# Slightly different part 1 solution

In [5]:
#Formatting
hands = getHands().split("\n")
cards = [hand.split()[0] for hand in hands]
bets = [int(hand.split()[1]) for hand in hands]
cardBets = dict(zip(cards, bets))

#Sort cards based on letter ordering
cardsOrder = "23456789TJQKA"
cards = sorted(cards, key=lambda card: [cardsOrder.find(x) for x in card]) #Sort by the index of cardsOrder

#Keep track of hand types
#0: high card, 1: one pair, 2: two pair, 3: three of a kind, 4: full house, 5: four of a kind, 6: five of a kind
handTypes = [[], [], [], [], [], [], []]

#Loop through each hand in cards
for hand in cards:
    
    #Checks for high card hand
    if len(set(hand)) == 5:
        handTypes[0].append(hand)
        continue
        
    #Checks for one pair
    if len(set(hand)) == 4:
        handTypes[1].append(hand)
        continue
        
    #Checks for two pair and three of a kind
    if len(set(hand)) == 3:
        #Check for three of a kind
        #If none of the first three cards have 3 occurrences in the hand, it is not three of a kind
        if hand.count(hand[0]) == 3 or hand.count(hand[1]) == 3 or hand.count(hand[2]) == 3:
            handTypes[3].append(hand)
            continue
        #Otherwise it is a two pair
        handTypes[2].append(hand)
        continue
    
    #Checks for full house and four of a kind
    if len(set(hand)) == 2:
        #Check for four of a kind
        #If either of the first two cards in a hand appear 4 times, then it is a four of a kind
        if hand.count(hand[0]) == 4 or hand.count(hand[1]) == 4:
            handTypes[5].append(hand)
            continue
        #Otherwise it's a full house
        handTypes[4].append(hand)
        continue
        
    #Checks for five of a kind
    if len(set(hand)) == 1:
        handTypes[6].append(hand)
        continue
        
#Flatten handTypes list as a new list: rankedHands
rankedHands = []
for i in handTypes:
    rankedHands.extend(i)

#Calculate total winnings
totalWinnings = 0
for rank, hand in enumerate(rankedHands, 1):
    totalWinnings += rank*cardBets[hand]
    
print(f"Total winnings: {totalWinnings}")

Total winnings: 250474325


# --- Part Two ---
To make things a little more interesting, the Elf introduces one additional rule. Now, J cards are jokers - wildcards that can act like whatever card would make the hand the strongest type possible.

To balance this, J cards are now the weakest individual cards, weaker even than 2. The other cards stay in the same order: A, K, Q, T, 9, 8, 7, 6, 5, 4, 3, 2, J.

J cards can pretend to be whatever card is best for the purpose of determining hand type; for example, QJJQ2 is now considered four of a kind. However, for the purpose of breaking ties between two hands of the same type, J is always treated as J, not the card it's pretending to be: JKKK2 is weaker than QQQQ2 because J is weaker than Q.

Now, the above example goes very differently:

`32T3K 765`

`T55J5 684`

`KK677 28`

`KTJJT 220`

`QQQJA 483`

- 32T3K is still the only one pair; it doesn't contain any jokers, so its strength doesn't increase.
- KK677 is now the only two pair, making it the second-weakest hand.
- T55J5, KTJJT, and QQQJA are now all four of a kind! T55J5 gets rank 3, QQQJA gets rank 4, and KTJJT gets rank 5.

With the new joker rule, the total winnings in this example are 5905.

Using the new joker rule, find the rank of every hand in your set. What are the new total winnings?

In [7]:
#Master plan:

#Sort by alphabet
#Sort into handTypes
    #If joker in hand: get best hand type for that hand
    #Otherwise, sort into lists regularly
#Flatten list
#Calculate score
#You are awesome!

In [8]:
import itertools

def getBestHandType(hand):
    """
    This function takes in a string: hand (5 characters representing each card)
    If there is a joker in the hand, it will get every combination that the hand can be with said jokers
    It goes through, and finds the highest ranking hand that the hand can be (if no jokers, then just the hand)
    Return: The highest ranking hand it can be
    """
    
    numJokers = hand.count("J")
    
    #If there are jokers in the hand
    if numJokers:
    
        #Check for known cases to avoid unnecessary work
        #Checks for 5 of a kind
        if numJokers == 5 or numJokers == 4:
            return "Five of a kind"
        #If there are 3 jokers and the other two cards are the same, then it's a 5 of a kind, otherwise, 4 of a kind
        if numJokers == 3 and hand.replace("J", "")[0] == hand.replace("J", "")[1]:
            return "Five of a kind"
        elif numJokers == 3:
            return "Four of a kind"

        #Get all possible hands with jokers
        hands = []
        #Loop through all combinations of joker replacements:
        for combos in itertools.combinations_with_replacement("23456789TQKA", numJokers):
            #Reset current hand so we can make a new hand with each combination
            currentHand = hand

            #Loop through each replacement in the combination
            for replacement in combos:
                #Replace the next "J" with the replacement
                currentHand = currentHand.replace("J", replacement, 1)
            #After all "J"s have been replaced, add it to the list of all hands
            hands.append(currentHand)
    
    #If there are no jokers in the hand, set hands = a list with just the original hand
    else:
        hands = [hand]
        
    
    #Get all hand types, and then return the highest valued type
    handTypes = []
    #Loop through hands:
    for hand in hands:
        
        #Checks for high card hand
        if len(set(hand)) == 5:
            handTypes.append("High card")
            continue
        
        #Checks for one pair
        if len(set(hand)) == 4:
            handTypes.append("One pair")
            continue
        
        #Checks for two pair and three of a kind
        if len(set(hand)) == 3:
            #Check for three of a kind
            #If none of the first three cards have 3 occurrences in the hand, it is not three of a kind
            if hand.count(hand[0]) == 3 or hand.count(hand[1]) == 3 or hand.count(hand[2]) == 3:
                handTypes.append("Three of a kind")
                continue
            #Otherwise it is a two pair
            handTypes.append("Two pair")
            continue
            
        #Check for full house and four of a kind
        if len(set(hand)) == 2:
            #Check for four of a kind
            #If either of the first two cards in a hand appear 4 times, then it is a four of a kind
            if hand.count(hand[0]) == 4 or hand.count(hand[1]) == 4:
                handTypes.append("Four of a kind")
                continue
            #Otherwise it is a full house
            handTypes.append("Full house")
            continue
            
        #If there is only 1 unique card, it is a five of a kind
        if len(set(hand)) == 1:
            return "Five of a kind"
    
    
    #Return the highest hand type in handTypes
    handTypeOrder = ["Five of a kind","Four of a kind","Full house","Three of a kind","Two pair","One pair","High card"]
    for i in handTypeOrder:
        if i in handTypes:
            return i
       
    
#Formatting
hands = getHands().split("\n")
cards = [x.split()[0] for x in hands]
bets = [int(x.split()[1]) for x in hands]
cardBets = dict(zip(cards, bets))

#Sort hands by custom card ranking
cardRanking = "J23456789TQKA"
cards = sorted(cards, key=lambda cards: [cardRanking.find(char) for char in cards])

#Keep track of hand types
handTypeKey={"High card": 0, "One pair": 1, "Two pair": 2, "Three of a kind": 3, "Full house": 4,
             "Four of a kind": 5, "Five of a kind": 6}
handTypes = [[], [], [], [], [], [], []]

#Loop through all cards
for hand in cards:
    #Get the best hand type
    bestHandType = getBestHandType(hand)
    #Append the hand to the list associated with the correct hand type
    handTypes[handTypeKey[bestHandType]].append(hand)

#Flatten handTypes into new list: rankedHands
rankedHands = []
for handType in handTypes:
    rankedHands.extend(handType)
    
#Calculate total winnings
totalWinnings = 0
for rank, hand in enumerate(rankedHands, 1):
    totalWinnings += rank * cardBets[hand]
    
print(f"Total winnings: {totalWinnings}")

Total winnings: 248909434
